In [ ]:
# -*- coding: utf-8 -*-
# UORA 메인 슬롯 루프 — 단계별(step-by-step) 실행/디버깅 가능한 코드
# run_one_slot(verbose=True)로 슬롯을 한 칸씩 진행하며 내부 상태를 확인할 수 있습니다.

import numpy as np
import math, time

# 난수
rng = np.random.default_rng(0)

# -------- 기본 파라미터 (원본과 동일/호환 값) --------
L_Assoc_Req_in_Slot = 5

# 시뮬/시간
N_slot_total = 20000
Observation_Period = 100
Opt_Observation_TH = True
Next_Observation_Time_TH = Observation_Period

# STA/입퇴장
N_STA = 20 #STA 수
N_STAs_existing = N_STA  # Mode 9용
N_unassoc_In  = 2 #비연관 STA 입장
N_unassoc_Out = 1 #비연관 STA 퇴장
T_Period_InOut = 500 #입퇴장 주기
Next_T_unassoc_In  = 200
Next_T_unassoc_Out = 400

# UORA/옵션
Mode_UORA = 0           # 0,2,3,4,5,6,7,8,9 중 선택
Mode_OBO_BEB = True
Mode_OBO_XIXD = False
Opt_Random_N_RA_RU_0 = False

# RU/OCW 테이블
N_RU = 16
N_RA_RU_0    = 8     # assoc용 랜덤접근 RU 수
N_RA_RU_2045 = 4     # unassoc용 랜덤접근 RU 수
RU_AID = np.zeros(N_RU, dtype=int)   # 0 또는 2045 등 AID 맵 (필요시 수정)

# Tab_OCW: 첫 열(col0)=STA 수, 나머지 열=RA-RU에 따른 OCW (예시 테이블)
rows = 64
cols = max(N_RA_RU_0, N_RA_RU_2045) + 2  # col0 + RU열들(1..)
Tab_OCW = np.zeros((rows, cols), dtype=float)
Tab_OCW[:,0] = np.linspace(0, 512, rows)  # STA 수 축
for c in range(1, cols):
    Tab_OCW[:,c] = 8.0 * (2**(c-1))       # 예시값: 8,16,32,...

def nearest_index_col1(tab, value):
    col0 = tab[:,0]
    return int(np.argmin(np.abs(col0 - value)))

# CW/alpha
CWmin, CWmax = 8.0, 1024.0
Init_alpha, Min_alpha, Max_alpha = 1.0, 0.05, 4.0
Inc, Dec = 0.10, 0.10
alpha_g = 1.0  # Mode 2/3용 글로벌 알파

# PHY/시간/레이트
L_PHY = 56.0
L_BACK = 14.0
L_Trigger = 48.0
SIFS = 16.0
T_dft = 3.2
T_gi  = 0.8
U_slot = 9.0
N_ss = 1
# RU별 N_sd, MCS별 비트/심볼 & 부호율
N_sd = np.full(N_RU, 52.0)
N_bp = np.array([1,2,2,4,6,6,8,8,10,10,12,12], dtype=float)  # 0..11
Rcod = np.array([1/2,1/2,3/4,1/2,2/3,3/4,2/3,3/4,3/4,5/6,3/4,5/6], dtype=float)

# 공통/배열 초기화
Common_MCS = 5  # 0-base index
STA_Type = np.zeros(N_STA, dtype=int)   # 0=Unassoc, 1=Assoc, -1=Disable
L_MPDU = np.full(N_STA, 2000.0)
MCS = np.full(N_STA, Common_MCS, dtype=int)
STA_CWmin = np.full(N_STA, CWmin, dtype=float)
STA_CWmax = np.full(N_STA, CWmax, dtype=float)
STA_CW    = np.full(N_STA, CWmin, dtype=float)
STA_OBO   = np.floor(STA_CW * rng.random(N_STA))
STA_Assigned_RU = np.zeros(N_STA, dtype=int)
STA_Assoc_Delay = np.zeros((N_STA,3), dtype=float)
Sim_Rst = np.zeros((N_STA,4), dtype=float)   # [Access,Success,Collisions,sum(n_tf)]
n_tf = np.zeros(N_STA, dtype=float)

# alpha/Succ (Mode 4~8)
alpha = np.full(N_STA, Init_alpha, dtype=float)
Succ = np.zeros((N_STA,2), dtype=float)      # [success_streak, collision_streak]

# RU 통계
Sim_Rst_RU = np.zeros((N_RU,3), dtype=float) # [IDLE,SUCCESS,COLLISION]

# 관측/로그
log_slot_time = []
log_alpha = []
log_slot_time_SUC = [0.0]    # MATLAB 누적 모사 위해 0으로 시작
log_SUC = [0.0]
Observation_N_assoc = []

# 루프 인덱스
i = 1
start_t = time.time()

# -----------------------------
# Step 1) 입/퇴장
# -----------------------------
def step_in_out():
    global N_STA, Next_T_unassoc_In, Next_T_unassoc_Out
    global STA_Type, L_MPDU, MCS, STA_CWmin, STA_CWmax, STA_CW, STA_OBO
    global STA_Assigned_RU, STA_Assoc_Delay, Sim_Rst, n_tf, alpha, Succ

    if Next_T_unassoc_In <= i and N_unassoc_In > 0:
        STA_ID_Disable = np.where(STA_Type == -1)[0]
        if len(STA_ID_Disable) == N_unassoc_In:
            # 재활성화
            STA_Type[STA_ID_Disable] = 0
            Next_T_unassoc_In = i + T_Period_InOut
        else:
            # 새로 추가
            old_N = N_STA
            N_STA += int(N_unassoc_In)
            Next_T_unassoc_In = i + T_Period_InOut

            STA_Type = np.concatenate([STA_Type, np.zeros(int(N_unassoc_In), dtype=int)])
            L_MPDU = np.concatenate([L_MPDU, np.full(int(N_unassoc_In), 2000.0)])
            MCS = np.concatenate([MCS, np.full(int(N_unassoc_In), Common_MCS, dtype=int)])
            STA_CWmin = np.concatenate([STA_CWmin, np.full(int(N_unassoc_In), CWmin, dtype=float)])
            STA_CWmax = np.concatenate([STA_CWmax, np.full(int(N_unassoc_In), CWmax, dtype=float)])
            STA_CW    = np.concatenate([STA_CW,    np.full(int(N_unassoc_In), CWmin, dtype=float)])
            STA_OBO   = np.concatenate([STA_OBO,   np.floor(CWmin * rng.random(int(N_unassoc_In)))])
            STA_Assigned_RU = np.concatenate([STA_Assigned_RU, np.zeros(int(N_unassoc_In), dtype=int)])
            add_delay = np.zeros((int(N_unassoc_In),3), dtype=float)
            add_delay[:,0] = (Next_T_unassoc_In - T_Period_InOut)
            STA_Assoc_Delay = np.vstack([STA_Assoc_Delay, add_delay])
            Sim_Rst = np.vstack([Sim_Rst, np.zeros((int(N_unassoc_In),4), dtype=float)])
            n_tf = np.concatenate([n_tf, np.zeros(int(N_unassoc_In), dtype=float)])

            if Mode_UORA in (4,5,6,7,8):
                alpha = np.concatenate([alpha, np.full(int(N_unassoc_In), Init_alpha, dtype=float)])
                if Mode_UORA in (5,6,7,8):
                    Succ = np.vstack([Succ, np.zeros((int(N_unassoc_In),2), dtype=float)])

    if Next_T_unassoc_Out <= i and N_unassoc_Out > 0:
        STA_ID_Assoc = np.where(STA_Type == 1)[0]
        Disable_STA_ID = []
        while len(Disable_STA_ID) < int(N_unassoc_Out) and len(STA_ID_Assoc) > 0:
            pick = rng.choice(STA_ID_Assoc)
            if pick not in Disable_STA_ID:
                Disable_STA_ID.append(pick)
                STA_Type[pick] = -1
        Next_T_unassoc_Out = i + T_Period_InOut

# -----------------------------
# Step 2) OBO 감소 (모드별)
# -----------------------------
def step_obo_decrement():
    global alpha_g
    # 현재 assoc/unassoc 집합
    STA_ID_Unassoc = np.where(STA_Type == 0)[0]
    STA_ID_Assoc   = np.where(STA_Type == 1)[0]

    if Mode_UORA in (0,9):
        if Opt_Random_N_RA_RU_0:
            N_Random_RA_RU_0 = int(math.ceil(rng.random()*N_RA_RU_0))
            if STA_ID_Unassoc.size:
                STA_OBO[STA_ID_Unassoc] -= N_RA_RU_2045
            if STA_ID_Assoc.size:
                STA_OBO[STA_ID_Assoc]   -= N_Random_RA_RU_0
        else:
            if STA_ID_Unassoc.size:
                STA_OBO[STA_ID_Unassoc] -= N_RA_RU_2045
            if STA_ID_Assoc.size:
                STA_OBO[STA_ID_Assoc]   -= N_RA_RU_0

    elif Mode_UORA == 1:
        raise RuntimeError('Mode_UORA == 1 not supported in provided script')

    elif Mode_UORA == 2:
        STA_OBO[:] -= alpha_g * N_RU

    elif Mode_UORA == 3:
        STA_OBO[:] -= alpha_g * N_RU

    elif Mode_UORA in (4,5,6,7,8):
        if STA_ID_Unassoc.size:
            STA_OBO[STA_ID_Unassoc] -= alpha[STA_ID_Unassoc] * N_RA_RU_2045
        if STA_ID_Assoc.size:
            NR = N_RA_RU_0
            STA_OBO[STA_ID_Assoc] -= alpha[STA_ID_Assoc] * NR

# -----------------------------
# Step 3) TF 발행 & RU 배정
# -----------------------------
def step_tf_and_ru_assign():
    # 반환: STA_ID_TX, STA_ID_TX_Unassoc, STA_ID_TX_Assoc
    STA_ID_Assoc   = np.where(STA_Type == 1)[0]
    STA_ID_TX = np.where(STA_OBO <= 0)[0]
    STA_ID_TX_Unassoc = np.setdiff1d(STA_ID_TX, STA_ID_Assoc, assume_unique=False)
    STA_ID_TX_Assoc   = np.setdiff1d(STA_ID_TX, STA_ID_TX_Unassoc, assume_unique=False)

    if STA_ID_TX_Assoc.size:
        if Opt_Random_N_RA_RU_0:
            N_Random_RA_RU_0 = max(1, int(math.ceil(rng.random()*N_RA_RU_0)))
            STA_Assigned_RU[STA_ID_TX_Assoc] = rng.integers(1, N_Random_RA_RU_0+1, size=STA_ID_TX_Assoc.size)
        else:
            STA_Assigned_RU[STA_ID_TX_Assoc] = rng.integers(1, N_RA_RU_0+1, size=STA_ID_TX_Assoc.size)
    if STA_ID_TX_Unassoc.size:
        STA_Assigned_RU[STA_ID_TX_Unassoc] = rng.integers(N_RA_RU_0+1, N_RU+1, size=STA_ID_TX_Unassoc.size)

    if STA_ID_TX.size:
        # n_tf 누적
        n_tf[:] = n_tf + (STA_OBO <= 0)
    return STA_ID_TX, STA_ID_TX_Unassoc, STA_ID_TX_Assoc

# -----------------------------
# Step 4) RU별 판정 (성공/충돌 & BEB/alpha 업데이트)
# -----------------------------
def step_ru_outcomes(STA_ID_TX):
    global alpha_g, STA_CW, Sim_Rst, Sim_Rst_RU, Succ
    STA_ID_SUC = []
    STA_ID_COL = []

    for j in range(1, N_RU+1):
        ids = np.where(STA_Assigned_RU == j)[0]
        if ids.size == 0:
            Sim_Rst_RU[j-1, 1-1] += 1  # IDLE
            continue

        Sim_Rst[ids, 0] += 1  # Access

        if ids.size > 1:
            # Collision
            Sim_Rst[ids, 2] += 1
            Sim_Rst_RU[j-1, 3-1] += 1
            STA_ID_COL.extend(ids.tolist())

            # BEB / XIXD / Mode 9 처리
            if Mode_OBO_BEB:
                if Mode_UORA == 9:
                    if RU_AID[j-1] == 0 and (N_unassoc_In > 0 or N_unassoc_Out > 0):
                        assoc_cnt = int(np.sum(STA_Type == 1))
                        r = nearest_index_col1(Tab_OCW, assoc_cnt)
                        col = (N_RA_RU_0 + 1)
                        col = min(col, Tab_OCW.shape[1]-1)
                        newcw = Tab_OCW[r, col]
                        STA_CW[ids] = newcw
                    elif RU_AID[j-1] == 2045 and (N_unassoc_In > 0 or N_unassoc_Out > 0):
                        unassoc_cnt = int(np.sum(STA_Type == 0))
                        r = nearest_index_col1(Tab_OCW, unassoc_cnt)
                        col = (N_RA_RU_2045 + 1)
                        col = min(col, Tab_OCW.shape[1]-1)
                        newcw = Tab_OCW[r, col]
                        STA_CW[ids] = newcw
                else:
                    # 기본 BEB
                    STA_CW[ids] = (STA_CW[ids] + 1.0) * 2.0 - 1.0
                    over = np.where(STA_CW[ids] >= STA_CWmax[ids])[0]
                    if over.size:
                        STA_CW[ids[over]] = STA_CWmax[ids[over]]

            # alpha 조정
            if Mode_UORA == 3:
                alpha_g = max(Min_alpha, alpha_g - Dec)
            elif Mode_UORA == 4:
                alpha[ids] = np.maximum(Min_alpha, alpha[ids] - Dec)
            elif Mode_UORA == 5:
                Succ[ids,1] += 1
                Succ[ids,0] = 0
                alpha[ids] = np.maximum(Min_alpha, alpha[ids] - Dec * (2.0 ** (Succ[ids,1]-1)))
            elif Mode_UORA == 6:
                for kk in ids:
                    Succ[kk,1] += 1
                    if Succ[kk,0] > 0:
                        alpha[kk] = Init_alpha
                    else:
                        alpha[kk] = max(Min_alpha, alpha[kk] - Dec)
                    Succ[kk,0] = 0
            elif Mode_UORA in (7,8):
                Succ[ids,1] += 1
                Succ[ids,0]  = 0
                if Mode_UORA == 7:
                    alpha[ids] = np.maximum(Min_alpha, alpha[ids] - Dec)
                else:
                    alpha[ids] = np.maximum(Min_alpha, alpha[ids] - Dec * (2.0 ** (Succ[ids,1]-1)))

        else:
            # Success
            one = ids[0]
            if STA_Type[one] == 1:
                Sim_Rst[one, 1] += 1
            Sim_Rst_RU[j-1, 2-1] += 1
            Sim_Rst[one, 3] += n_tf[one]
            n_tf[one] = 0
            STA_ID_SUC.append(one)

            # 성공 시 CW 갱신
            if Mode_OBO_BEB:
                if Mode_UORA == 9:
                    if RU_AID[j-1] == 0 and (N_unassoc_In > 0 or N_unassoc_Out > 0):
                        assoc_cnt = int(np.sum(STA_Type == 1))
                        r = nearest_index_col1(Tab_OCW, assoc_cnt)
                        col = (N_RA_RU_0 + 1)
                        col = min(col, Tab_OCW.shape[1]-1)
                        newcw = Tab_OCW[r, col]
                        STA_CW[one] = newcw
                    elif RU_AID[j-1] == 2045 and (N_unassoc_In > 0 or N_unassoc_Out > 0):
                        unassoc_cnt = int(np.sum(STA_Type == 0))
                        r = nearest_index_col1(Tab_OCW, unassoc_cnt)
                        col = (N_RA_RU_2045 + 1)
                        col = min(col, Tab_OCW.shape[1]-1)
                        newcw = Tab_OCW[r, col]
                        STA_CW[one] = newcw
                else:
                    STA_CW[one] = STA_CWmin[one]

            # alpha 증가
            if Mode_UORA == 3:
                alpha_g = min(Max_alpha, alpha_g + Inc)
            elif Mode_UORA == 4:
                alpha[one] = min(Max_alpha, alpha[one] + Inc)
            elif Mode_UORA == 5:
                Succ[one,0] += 1
                Succ[one,1]  = 0
                alpha[one] = min(Max_alpha, alpha[one] + Inc * (2.0 ** (Succ[one,0]-1)))
            elif Mode_UORA == 6:
                Succ[one,0] += 1
                if Succ[one,1] > 0:
                    alpha[one] = Init_alpha + Inc
                else:
                    alpha[one] = min(Max_alpha, alpha[one] + Inc)
                Succ[one,1] = 0
            elif Mode_UORA in (7,8):
                Succ[one,0] += 1
                Succ[one,1]  = 0
                if Mode_UORA == 7:
                    alpha[one] = min(Max_alpha, alpha[one] + Inc * (2.0 ** (Succ[one,0]-1)))
                else:
                    alpha[one] = min(Max_alpha, alpha[one] + Inc)
    return STA_ID_SUC, STA_ID_COL

# -----------------------------
# Step 5) 시간 전진 & Assoc 성공 처리
# -----------------------------
def step_time_advance_and_assoc_success(STA_ID_TX, STA_ID_SUC):
    global i, STA_Assigned_RU, STA_Type, STA_Assoc_Delay

    if STA_ID_TX.size:
        # 데이터 레이트 계산
        data_rates = np.zeros(STA_ID_TX.size, dtype=float)
        for idx_k, ksta in enumerate(STA_ID_TX):
            ru = STA_Assigned_RU[ksta]
            ru = max(1, min(N_RU, ru))
            mcs = MCS[ksta]
            data_rates[idx_k] = (N_sd[ru-1] * N_bp[mcs] * Rcod[mcs] * N_ss) / (T_dft + T_gi)

        STA_ID_assoc_req = STA_ID_TX[STA_Type[STA_ID_TX] == 0]
        STA_ID_Data_TX   = STA_ID_TX[STA_Type[STA_ID_TX] == 1]

        # 시간 소모
        if STA_ID_assoc_req.size and STA_ID_Data_TX.size:
            dur = int( np.max( np.ceil( (L_PHY + (8*L_MPDU[STA_ID_TX]/data_rates)) / U_slot ) ) ) \
                  + int( np.ceil( (2*L_PHY + L_BACK + L_Trigger + 3*SIFS) / U_slot) )
            i += dur
        elif STA_ID_assoc_req.size:
            dur = int(L_Assoc_Req_in_Slot) \
                  + int( np.ceil( (2*L_PHY + L_BACK + L_Trigger + 3*SIFS) / U_slot ) )
            i += dur
        elif STA_ID_Data_TX.size:
            dur = int( np.max( np.ceil( (L_PHY + (8*L_MPDU[STA_ID_TX]/data_rates)) / U_slot ) ) ) \
                  + int( np.ceil( (2*L_PHY + L_BACK + L_Trigger + 3*SIFS) / U_slot) )
            i += dur
        else:
            raise RuntimeError('unexpected situation!')

        # Assoc 성공 처리
        for ksta in STA_ID_assoc_req:
            if ksta in STA_ID_SUC:
                STA_Type[ksta] = 1
                ack_time = i - int(np.ceil((2*L_PHY + L_BACK + L_Trigger + 3*SIFS) / U_slot))
                STA_Assoc_Delay[ksta,1] = ack_time
                STA_Assoc_Delay[ksta,2] = STA_Assoc_Delay[ksta,1] - STA_Assoc_Delay[ksta,0]

        # reset
        STA_Assigned_RU[:] = 0
        return True  # TF 있었음
    else:
        i += 1
        return False  # TF 없음

# -----------------------------
# Step 6) 관측 업데이트 & 루프 말미 처리
# -----------------------------
def step_observation_and_tail(TF_happened):
    global Next_Observation_Time_TH, log_slot_time_SUC, log_SUC

    # 관측
    Observation_N_assoc.append([i,
                                float(np.sum(STA_Type == 0)),
                                float(np.sum(STA_Type == 1))])

    # Throughput 관측 주기 업데이트
    if Opt_Observation_TH and (Next_Observation_Time_TH is not None) and (Next_Observation_Time_TH <= i):
        log_slot_time_SUC.append(i - sum(log_slot_time_SUC))
        log_SUC.append( float(np.sum(Sim_Rst[:,1])) - sum(log_SUC) )
        Next_Observation_Time_TH += Observation_Period

    # 루프 말미 옵션: 랜덤 RA-RU 및 Mode9 CW 재적용
    if Opt_Random_N_RA_RU_0:
        N_Random_RA_RU_0 = int(math.ceil(rng.random()*N_RA_RU_0))
        if Mode_UORA == 9 and N_STAs_existing > 0:
            r = nearest_index_col1(Tab_OCW, N_STAs_existing)
            col = (N_Random_RA_RU_0 + 1)
            col = min(col, Tab_OCW.shape[1]-1)
            newcw = Tab_OCW[r, col]
            STA_CW[:N_STAs_existing] = newcw

    # TF 있었던 STA들의 OBO 재시작
    if TF_happened:
        STA_ID_TX = np.where(STA_OBO <= 0)[0]
        if STA_ID_TX.size:
            STA_OBO[STA_ID_TX] = np.floor(STA_CW[STA_ID_TX] * rng.random(STA_ID_TX.size))

# -----------------------------
# 한 슬롯 실행 (디버그 출력 옵션)
# -----------------------------
def run_one_slot(verbose=True):
    # 1) 입/퇴장
    step_in_out()
    # 2) OBO 감소
    step_obo_decrement()
    # 3) TF & RU 배정
    STA_ID_TX, _, _ = step_tf_and_ru_assign()
    # 4) RU별 판정
    STA_ID_SUC, STA_ID_COL = step_ru_outcomes(STA_ID_TX)
    # 5) 시간 전진 & Assoc 성공 처리
    TF_happened = step_time_advance_and_assoc_success(STA_ID_TX, STA_ID_SUC)
    # 6) 관측 업데이트 & 루프 말미 처리
    step_observation_and_tail(TF_happened)

    if verbose:
        print(f"i={i}, Unassoc={np.sum(STA_Type==0):.0f}, Assoc={np.sum(STA_Type==1):.0f}, "
              f"TX={STA_ID_TX.size}, S={len(STA_ID_SUC)}, C={len(STA_ID_COL)}")

# -----------------------------
# 예시: 몇 슬롯만 단계별로 실행
# -----------------------------
if __name__ == "__main__":
    for _ in range(5):
        run_one_slot(verbose=True)

    print("--- 상태 요약 ---")
    print("i:", i)
    print("N_STA:", N_STA)
    print("최근 Observation_N_assoc:", Observation_N_assoc[-3:])


i=31, Unassoc=17, Assoc=3, TX=11, S=3, C=8
i=93, Unassoc=15, Assoc=5, TX=13, S=5, C=8
i=155, Unassoc=10, Assoc=10, TX=10, S=8, C=2
i=217, Unassoc=9, Assoc=11, TX=15, S=4, C=11
i=279, Unassoc=9, Assoc=13, TX=7, S=5, C=2
--- 상태 요약 ---
i: 279
N_STA: 22
최근 Observation_N_assoc: [[155, 10.0, 10.0], [217, 9.0, 11.0], [279, 9.0, 13.0]]
